In [60]:
import os
import numpy as np
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.warp import calculate_default_transform, reproject
from rasterio.enums import Resampling

In [61]:
shp = gpd.read_file("E:/NTNU/urban_gis/project/data/study_area/study_area_v3.shp")
geo = shp.geometry
bands = ["B4", "B5", "B6"]

rasterfilePath = "E:/NTNU/urban_gis/project/data"

rasterfileName = "LC09_L1TP_117043_20230725_20230725_02_T1"
year = rasterfileName[17:21]

r = f"{rasterfilePath}/{rasterfileName}/{rasterfileName}_B4.TIF"
nir = f"{rasterfilePath}/{rasterfileName}/{rasterfileName}_B5.TIF"
swir = f"{rasterfilePath}/{rasterfileName}/{rasterfileName}_B6.TIF"

output_folder = f"{rasterfilePath}/proj/{rasterfileName}"
os.makedirs(output_folder, exist_ok=True)

In [62]:
for band in bands:
    input_path = f"{rasterfilePath}/{rasterfileName}/{rasterfileName}_{band}.TIF"
    output_path = f"{output_folder}/{rasterfileName}_{band}.TIF"

    with rasterio.open(input_path) as src:
        transform, width, height = calculate_default_transform(
            src.crs, 'EPSG:3826', src.width, src.height, *src.bounds)
        
        kwargs = src.meta.copy()
        kwargs.update({
            'crs': 'EPSG:3826',
            'transform': transform,
            'width': width,
            'height': height
        })

        with rasterio.open(output_path, 'w', **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs='EPSG:3826',
                    resampling=Resampling.nearest)

In [63]:
def clip_raster(raster_path, shapes):
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, shapes, crop=True)
        out_meta = src.meta.copy()
        out_meta.update({"driver": "GTiff",
                         "height": out_image.shape[1],
                         "width": out_image.shape[2],
                         "transform": out_transform})
    return out_image, out_meta

for band in bands:
    raster_path = f"{output_folder}/{rasterfileName}_{band}.TIF"
    clipped, meta = clip_raster(raster_path, geo)

    with rasterio.open(f"proj/{rasterfileName}_{band}.tif", "w", **meta) as dest:
        dest.write(clipped)

In [64]:
def calculate_ndvi_ndbi(r, nir, swir):
    with rasterio.open(r) as red_src:
        red = red_src.read(1).astype('float32')

    with rasterio.open(nir) as nir_src:
        nir = nir_src.read(1).astype('float32')

    with rasterio.open(swir) as swir_src:
        swir = swir_src.read(1).astype('float32')

    ndvi = (nir - red) / (nir + red)
    ndbi = (swir - nir) / (swir + nir)

    return ndvi, ndbi

def save_tiff(data, file_path, meta):
    with rasterio.open(file_path, "w", **meta) as dest:
        dest.write(data, 1)

r = f"proj/{rasterfileName}_B4.tif"
nir = f"proj/{rasterfileName}_B5.tif"
swir = f"proj/{rasterfileName}_B6.tif"

ndvi, ndbi = calculate_ndvi_ndbi(r, nir, swir)

save_tiff((ndvi + 1) * 32768, f"ndvi/{year}_NDVI.tif", meta)
save_tiff((ndbi + 1) * 32768, f"ndbi/{year}_NDBI.tif", meta)

C:\Users\Chang Tzu Cheng\AppData\Local\Temp\ipykernel_26492\4275525968.py:11: RuntimeWarning: invalid value encountered in divide
  ndvi = (nir - red) / (nir + red)
C:\Users\Chang Tzu Cheng\AppData\Local\Temp\ipykernel_26492\4275525968.py:12: RuntimeWarning: invalid value encountered in divide
  ndbi = (swir - nir) / (swir + nir)
c:\ProgramData\anaconda3\envs\arcpy\lib\site-packages\numpy\core\_asarray.py:126: RuntimeWarning: invalid value encountered in cast
  arr = array(a, dtype=dtype, order=order, copy=False, subok=subok)
